In [6]:
import pandas as pd
import numpy as np
import re
import pymysql
import datetime
from datetime import timedelta
from sqlalchemy import create_engine

In [3]:
# 数据库信息
mysql_setting = {
    'host': '47.100.201.211',
    'port': 3306,
    'user': 'root',
    'passwd': 'iyGfLR64Ne4Ddhk7',
    # 数据库名称
    'db': 'data',
    'charset': 'utf8'
}
engine = create_engine("mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}".format(**mysql_setting), max_overflow=5)

# getRealPrice

In [17]:
# 填充缺失价格，去重重复天数价格
def fillPrice(df):
    # datetime.strptime() 把datetime转化为str格式
    df['updateTime'] = df['updateTime'].apply(lambda x: x.strftime('%Y-%m-%d'))
    date_start = df['updateTime'].min()
    date_end = datetime.datetime.now().strftime('%Y-%m-%d')
    #根据最小日期和最大日期，计算日期间隔，由于date_start和date_end为string类型，因此需要先更改为日期类型
    delta = datetime.datetime.strptime(date_end, "%Y-%m-%d")-datetime.datetime.strptime(str(date_start), "%Y-%m-%d")
    for i in range(1,delta.days+1):
        date = (datetime.datetime.strptime(date_end, "%Y-%m-%d")-datetime.timedelta(days=i)).strftime('%Y-%m-%d')
        #如果数据缺失，则补数
        if(date not in df['updateTime'].values):
            new_date = pd.DataFrame({'updateTime':date,'dayPrice':np.nan}, index=[0])
            df = df.append(new_date)
    df.sort_values(by="updateTime", inplace=True, ascending=True) 
    df['dayPrice'] = df['dayPrice'].fillna(df['dayPrice'].interpolate().round(2))
    df['updateTime'] = pd.to_datetime(df['updateTime'])
    df['updateTime'] = df['updateTime'].dt.strftime("%Y-%m")
    df = df.groupby('updateTime').mean('dayPrice').round(2)
    df.reset_index(level=0, inplace=True)
    return df

In [18]:
def getRealPrice(product, url, w):
    if ('jd' in url):
        sql_cmd = "SELECT updateTime, dayPrice FROM data.jingdong_day_price_nineCounty where commodityUrl = %(url)s"
    else:
        sql_cmd = "SELECT updateTime, dayPrice FROM data.tianmao_day_price_nineCounty where commodityId = %(url)s"
    df = pd.read_sql(sql_cmd, engine, params={'url': url})
    df.drop(index=df[df['dayPrice']<=0].index,inplace=True)
    df.drop_duplicates(subset='updateTime', keep='last', inplace=True)
    df = fillPrice(df)
    df['dayPrice'] = df['dayPrice'].apply(lambda x: (x / w))
    df['dayPrice'] = df['dayPrice'].round(2)
    df['product'] = product
    df.rename(columns={'dayPrice': 'real_price', 'updateTime': 'date'}, inplace=True)
    newIndex = ['product', 'date', 'real_price']
    df = df[newIndex]
    return df

In [19]:
result = engine.execute('TRUNCATE TABLE data.price_real_long')
data = pd.read_csv('allProduct.csv', encoding='utf-8')
for row in data.iterrows():
    try:
        table = getRealPrice(row[1]['product'], row[1]['url'], row[1]['weight'])
        table.to_sql('price_real_long', con= engine, if_exists= 'append', index=False)
    except:
        print(row[1]['product'])

In [8]:
cc = "2022-08-11"
dd = "2022-11"

dd.length()

AttributeError: 'str' object has no attribute 'length'

In [5]:
# aa = getRealPrice('奉新大米', 'https://detail.tmall.com/item.htm?spm=a230r.1.14.16.25333797FZSHM9&id=610942185103&ns=1&abbucket=5', 10)
# aa

,product,date,real_price
0,奉新大米,2022-04,6.800
1,奉新大米,2022-05,6.800
2,奉新大米,2022-06,6.800
3,奉新大米,2022-07,6.800
4,奉新大米,2022-08,11.422


# comparePrice

In [5]:
# 清空price_real_predict_compare_short表
# def comparePrice(product):
#     month = datetime.datetime.now().strftime('%Y-%m')
#     sql_cmd = "SELECT * FROM price_real_long where product = %(product)s"
#     aa = pd.read_sql(sql_cmd, engine, params={'product': product})
#     sql_cmd = "SELECT * FROM price_predict_long where product = %(product)s and date = %(month)s"
#     bb = pd.read_sql(sql_cmd, engine, params={'product': product, 'month': month})
#     bb.drop(columns='date', inplace = True)
#     bb.rename(columns={'predict_date': 'date'}, inplace=True)
#     compareP = pd.merge(aa, bb, on=['product','date'], how='outer')
#     compareP['predict_price'] = compareP['predict_price'].round(2)
#     compareP['real_price'] = compareP['real_price'].round(2)
#     return compareP
# result = engine.execute('TRUNCATE TABLE data.price_real_predict_compare_long')
# data = pd.read_csv('allProduct.csv', encoding='utf-8')
# for row in data.iterrows():
#     try:
#         table = comparePrice(row[1]['product'])
#         table.to_sql('price_real_predict_compare_long', con= engine, if_exists= 'append', index=False)
#     except:
#         print(row[1]['product'])

In [10]:
# def insertNew(product):
#     sql_cmd = "SELECT * FROM price_real_predict_compare_short where product = %(product)s"
#     aa = pd.read_sql(sql_cmd, engine, params={'product': product})
#     # 获取当天时间
#     lastday = datetime.datetime.now().strftime('%Y-%m-%d')
#     day = datetime.datetime.date(datetime.datetime.strptime(lastday, "%Y-%m-%d"))
    
#     sql_cmd = "SELECT * FROM price_real_short where product = %(product)s and date = %(date)s"
#     rr = pd.read_sql(sql_cmd, engine, params={'product': product, 'date': lastday})
#     aa.loc[aa['date']==day, 'real_price'] = rr['real_price'].values[0]
    
#     # 获取当天的实际价格
#     params = {'price': rr['real_price'].values[0], 'product': product, 'day': lastday}
#     sql_cmd = "UPDATE price_real_predict_compare_short SET real_price = %(price)s WHERE product = %(product)s and date = %(day)s "
#     res = engine.execute(sql_cmd, params)
    
#     # 获取最新预测的价格，并且和之前的价格表进行concat上下连接
#     new = (datetime.datetime.now() + datetime.timedelta(days=14)).strftime('%Y-%m-%d')
#     sql_cmd = "SELECT * FROM price_predict_short where product = %(product)s and predict_date = %(day)s"
#     bb = pd.read_sql(sql_cmd, engine, params={'product': product, 'day': new})
#     bb.drop(columns='date', inplace = True)
#     bb.rename(columns={'predict_date': 'date'}, inplace=True)
#     cc = pd.concat([aa,bb])
#     return cc[-1:]

In [11]:
# data = pd.read_csv('allProduct.csv', encoding='utf-8')
# for row in data.iterrows():
#     try:
#         table = insertNew(row[1]['product'])
#         table.to_sql('price_real_predict_compare_short', con= engine, if_exists= 'append', index=False)
        
#     except:
#         print(row[1]['product'])

四川宜宾屏山茵红李
贵州镇宁蜂糖李
